# Project ClearChoise Textile Classifier


#### The aim of this project is to develop a pattern classifier. By inputting an image, the program will provide feedback on the pattern present in the photographed garment. Making mornings easier and more fun for those with visual impairments, helping them pick out their perfect outfit effortlessly!

- Model for flowers detection. flower_model_28-02-24.pt
- Model for solid colors. pattern_model_24-03-06.h5
- Model for jeans or not. jeans_model_24-03-06.h5
- Model for geometric patterns. geometric_24-03-04.h5

In [28]:
import cv2
import numpy as np
import torch
import tensorflow as tf
torch.__version__

'2.2.1+cpu'

If there is problem with yolo locally:

In [29]:
import pathlib
from pathlib import Path
pathlib.PosixPath = pathlib.WindowsPath

## Load Flowermodel

In [30]:
yolo_path = "C:/Users/emmel/Desktop/yolov5"
model_path = "./Flower_model/flower_model_14-03-24.pt"


yolo_model = torch.hub.load(yolo_path, 'custom', path=model_path, source='local')
yolo_model.conf = 0.30

YOLOv5  v6.1-362-g731a2f8c Python-3.10.6 torch-2.2.1+cpu CPU



Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients
Adding AutoShape... 


## Load Patterns or not model

In [31]:
patterns_modelpath = "./solid_color_models/pattern_model_24-03-14.h5" 

patterns_model = tf.keras.models.load_model(patterns_modelpath)

## Load Jeansmodel

In [32]:
jeans_modelpath = "./solid_color_models/jeans_model_24-03-06.h5" 

jeans_model = tf.keras.models.load_model(jeans_modelpath)

labels_jeans = ["Not jeans", 
                "Jeans", 
]

## Load Geometricmodel

In [33]:
geometric_modelpath = "./geometric_model/geometric_24-03-27.h5" 

geometric_model = tf.keras.models.load_model(geometric_modelpath)

labels_geometric = ["check", 
                    "stripe", 
                    "dots"
]

### Test Images

In [34]:
img_paths = [ "./img_for_test/stripe_1.jpg",
                "./img_for_test/stripe_2.jpg",
                "./img_for_test/stripe_3.jpg",
                "./img_for_test/check_1.jpg",
                "./img_for_test/check_2.jpg",
                "./img_for_test/check_3.jpg",
                "./img_for_test/check_4.jpg",
                "./img_for_test/color_1.jpg",
                "./img_for_test/color_2.jpg",
                "./img_for_test/color_3.jpg",
                "./img_for_test/dots_1.jpg",
                "./img_for_test/dots_2.jpg",
                "./img_for_test/flowers_1.jpg",
                "./img_for_test/flowers_2.jpg",
                "./img_for_test/flowers_3.jpeg",
                "./img_for_test/flowers_4.jpg",
                "./img_for_test/jeans_1.jpg",
                "./img_for_test/jeans_2.jpg",
                ]

### Functions

In [35]:
def load_and_preprocess_image(image_path):
    img_dim = (64,64)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, img_dim)
    img = img[:, :, np.newaxis]
    img = img / 255
    return np.expand_dims(img, axis=0)

def predict_pattern_keras(img_preprocessed):
    pred_pattern = patterns_model.predict(img_preprocessed)
    pred_class_pattern = np.argmax(pred_pattern)

    if pred_class_pattern == 1:  # solid color
        predictions_jeans = jeans_model.predict(img_preprocessed)
        predicted_class_jeans = np.argmax(predictions_jeans)
        return labels_jeans[predicted_class_jeans]
        
    elif pred_class_pattern == 0:
        pred_geometric = geometric_model.predict(img_preprocessed)
        pred_class_geometric = np.argmax(pred_geometric)
        return labels_geometric[pred_class_geometric]
    else:
        return "Unknown Pattern"

# Function to predict pattern for images
def predict_pattern_for_images(img_paths):
    for img_path in img_paths:
        img = cv2.imread(img_path)
        yolo_result = yolo_model(img)
        yolo_preds = yolo_result.pandas().xyxy[0]
        
        if len(yolo_preds) > 0:
            print(f"{img_path}\nPredicted pattern: Flowers\n")
        else:
            img_preprocessed = load_and_preprocess_image(img_path)
            prediction = predict_pattern_keras(img_preprocessed)
            print(f"{img_path}\nPredicted pattern: {prediction}\n")

### Run

In [36]:
predict_pattern_for_images(img_paths)

1/1 [==============================] - 0s 89ms/step
./img_for_test/stripe_1.jpg
Predicted pattern: stripe

1/1 [==============================] - 0s 24ms/step
./img_for_test/stripe_2.jpg
Predicted pattern: stripe

1/1 [==============================] - 0s 55ms/step
./img_for_test/stripe_3.jpg
Predicted pattern: Jeans

1/1 [==============================] - 0s 24ms/step
./img_for_test/check_1.jpg
Predicted pattern: check

./img_for_test/check_2.jpg
Predicted pattern: Flowers

1/1 [==============================] - 0s 22ms/step
./img_for_test/check_3.jpg
Predicted pattern: check

1/1 [==============================] - 0s 21ms/step
./img_for_test/check_4.jpg
Predicted pattern: stripe

1/1 [==============================] - 0s 20ms/step
./img_for_test/color_1.jpg
Predicted pattern: Not jeans

1/1 [==============================] - 0s 21ms/step
./img_for_test/color_2.jpg
Predicted pattern: Not jeans

1/1 [==============================] - 0s 21ms/step
./img_for_test/color_3.jpg
Predicted pa

In [41]:
# Define color names and their corresponding RGB values
color_names = {
    'black': (0, 0, 0),
    'white': (255, 255, 255),
    'red': (255, 0, 0),
    'green': (0, 128, 0),
    'blue': (0, 0, 255),
    #'yellow': (255,255,204), #ljusgul
    #'orange': (255,165,0),
    #'purple': (153,50,204)
}

# Function to calculate color difference
def color_difference(rgb1, rgb2):
    rmean = (rgb1[0] + rgb2[0]) / 2
    r = rgb1[0] - rgb2[0]
    g = rgb1[1] - rgb2[1]
    b = rgb1[2] - rgb2[2]
    return np.sqrt((2 + rmean / 256) * r ** 2 + 4 * g ** 2 + (2 + (255 - rmean) / 256) * b ** 2)

# Function to get the closest color
def get_color_name(rgb_tuple):
    closest_color_name = None
    min_distance = float('inf')
    for name, rgb in color_names.items():
        distance = color_difference(rgb_tuple, rgb)
        if distance < min_distance:
            min_distance = distance
            closest_color_name = name
    return closest_color_name

def output_color(img_paths):
    image = cv2.cvtColor(cv2.imread(img_paths), cv2.COLOR_BGR2RGB)
    unique_colors, counts = np.unique(image.reshape(-1, 3), axis=0, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_colors = unique_colors[sorted_indices]
    top_color_names = [get_color_name(tuple(color)) for color in sorted_colors[:1]]
    return top_color_names

In [38]:
# for img in img_paths:
#     color = output_color(img) 
#     print (color)

In [42]:
# Function to predict pattern for images
def predict_pattern_for_images(img_paths):
    for img_path in img_paths:
        img_color = output_color(img_path)
        img = cv2.imread(img_path)
        yolo_result = yolo_model(img)
        yolo_preds = yolo_result.pandas().xyxy[0]
        
        if len(yolo_preds) > 0:
            print(f"{img_path}\nPredicted pattern: Flowers")
            print(f"In color: {img_color[0]}\n")
        else:
            img_preprocessed = load_and_preprocess_image(img_path)
            prediction = predict_pattern_keras(img_preprocessed)
            print(f"{img_path}\nPredicted pattern: {prediction}")
            print(f"In color: {img_color[0]}\n")

In [43]:
predict_pattern_for_images(img_paths)

1/1 [==============================] - 0s 22ms/step
./img_for_test/stripe_1.jpg
Predicted pattern: stripe
In color: white

1/1 [==============================] - 0s 21ms/step
./img_for_test/stripe_2.jpg
Predicted pattern: stripe
In color: white

1/1 [==============================] - 0s 23ms/step
./img_for_test/stripe_3.jpg
Predicted pattern: Jeans
In color: white

1/1 [==============================] - 0s 24ms/step
./img_for_test/check_1.jpg
Predicted pattern: check
In color: white

./img_for_test/check_2.jpg
Predicted pattern: Flowers
In color: white

1/1 [==============================] - 0s 24ms/step
./img_for_test/check_3.jpg
Predicted pattern: check
In color: white

1/1 [==============================] - 0s 22ms/step
./img_for_test/check_4.jpg
Predicted pattern: stripe
In color: green

1/1 [==============================] - 0s 22ms/step
./img_for_test/color_1.jpg
Predicted pattern: Not jeans
In color: white

1/1 [==============================] - 0s 22ms/step
./img_for_test/color